In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
tracts = gpd.read_file("data/analysis/hard-to-count-tracts.geojson")

In [3]:
tracts.head()

,geoid,state,county,pop,mrr,mrr_htc,ue,htc,geometry
0,01005950300,Alabama,Barbour County,1813,77.9,0,0,0,POLYGON ((-85.52744143500958 31.86650898512998...
1,01005950900,Alabama,Barbour County,3888,74.6,0,0,0,POLYGON ((-85.16412632985951 31.83060098962552...
2,01005950800,Alabama,Barbour County,2157,83.5,0,0,0,POLYGON ((-85.14872232914963 31.90934100751823...
3,01005950700,Alabama,Barbour County,1775,79.6,0,0,0,POLYGON ((-85.14578832747586 31.89149700368442...
4,01005950600,Alabama,Barbour County,2120,79.4,0,0,0,"POLYGON ((-85.14572732739356 31.8901120033811,..."


In [4]:
len(tracts)

72845

In [5]:
htc = tracts[tracts.htc == 1]

In [6]:
len(htc)

14793

In [73]:
htc.to_file("./data/analysis/htc-tracts.shp")

In [7]:
tribal_lands = gpd.read_file("data/tiger/tl_2018_us_aiannh/tl_2018_us_aiannh.shp")

In [8]:
trimmed_tribal = tribal_lands[['AIANNHCE', 'NAMELSAD', 'geometry']]

In [9]:
prepped_tribal = trimmed_tribal.rename(columns={
    "AIANNHCE": "geoid_tribal",
    "NAMELSAD": "name_tribal",
})

In [10]:
prepped_tribal.head()

,geoid_tribal,name_tribal,geometry
0,2320,Mohegan Reservation,"(POLYGON ((-72.089125 41.480395, -72.089004 41..."
1,9100,Golden Hill Paugussett (state) Reservation,"(POLYGON ((-72.26917299999999 41.554725, -72.2..."
2,4110,Table Mountain Rancheria,"POLYGON ((-119.640915 36.984246, -119.640807 3..."
3,1380,Greenville Rancheria,"POLYGON ((-120.897365 40.150998, -120.897334 4..."
4,1110,Flathead Reservation,"POLYGON ((-114.851209 47.887207, -114.819894 4..."


In [11]:
def get_geotype(geoid):
    val = int(geoid)
    if val < 5000:
        return 'Federal reservation or trust land'
    elif val < 5500:
        return 'Hawaiian home land'
    elif val < 6000:
        return 'Oklahoma tribal statistical area'
    elif val < 8000:
        return 'Alaska native village statistical area'
    elif val < 9000:
        return 'Tribal-designated statistical area'
    elif val < 9500:
        return 'State reservation'
    elif val < 9999:
        return 'State-designated tribal statistical area'

In [12]:
prepped_tribal['geotype'] = prepped_tribal.geoid_tribal.apply(get_geotype)

In [13]:
prepped_tribal.geotype.value_counts(dropna=False)

Federal reservation or trust land           477
Alaska native village statistical area      220
Hawaiian home land                           75
State-designated tribal statistical area     30
Oklahoma tribal statistical area             29
State reservation                            10
Tribal-designated statistical area            4
Name: geotype, dtype: int64

In [34]:
prepped_tribal.sort_values("area", ascending=False).head(10)

,geoid_tribal,name_tribal,geometry,geotype,area
117,2430,Navajo Nation Reservation,"(POLYGON ((-108.608106 35.587676, -108.606511 ...",Federal reservation or trust land,57664.349316
781,5590,Choctaw OTSA,"POLYGON ((-96.514343 34.488516, -96.513891 34....",Oklahoma tribal statistical area,28195.941633
798,5560,Cheyenne-Arapaho OTSA,"POLYGON ((-100.000419 35.204752, -100.000419 3...",Oklahoma tribal statistical area,21176.603213
780,5580,Chickasaw OTSA,"POLYGON ((-98.00044299999999 35.334304, -98.00...",Oklahoma tribal statistical area,19317.319432
595,6785,Knik ANVSA,"POLYGON ((-151.845385 62.288083, -151.843782 6...",Alaska native village statistical area,19076.818003
783,5550,Cherokee OTSA,"POLYGON ((-96.001561 36.321552, -96.001561 36....",Oklahoma tribal statistical area,18034.587105
252,4390,Uintah and Ouray Reservation,"POLYGON ((-111.312183 40.288136, -111.312031 4...",Federal reservation or trust land,17670.928264
789,5720,Kiowa-Comanche-Apache-Fort Sill Apache OTSA,"POLYGON ((-100.000479 34.708601, -100.000397 3...",Oklahoma tribal statistical area,16650.893389
594,6290,Chickaloon ANVSA,"POLYGON ((-149.47215 62.442173, -149.471613 62...",Alaska native village statistical area,15586.783360
788,5620,Creek OTSA,"POLYGON ((-96.624883 35.464302, -96.624832 35....",Oklahoma tribal statistical area,12395.791897


In [31]:
reservations = prepped_tribal[prepped_tribal.geotype == 'Federal reservation or trust land']

In [43]:
reservations_deduped = reservations.dissolve(by='geoid_tribal').reset_index()

In [45]:
reservations_deduped['area'] = reservations_deduped.to_crs({'proj':'cea'}).geometry.map(lambda p: p.area / 10**6)

In [32]:
reservations_reprojected = reservations_deduped.to_crs({'init': 'epsg:4326'})

In [48]:
intersection = gpd.overlay(reservations_reprojected, htc, how='intersection')

In [49]:
intersection.head()

,geoid_tribal,name_tribal,geotype,area,geoid,state,county,pop,mrr,mrr_htc,ue,htc,geometry
0,1110,Flathead Reservation,Federal reservation or trust land,5329.733273,30047940700,Montana,Lake County,2341,99999.0,0,1,1,POLYGON ((-113.8923431235792 47.30843603761106...
1,1110,Flathead Reservation,Federal reservation or trust land,5329.733273,30047940600,Montana,Lake County,4100,99999.0,0,1,1,POLYGON ((-113.8876667485412 47.36477716001926...
2,1110,Flathead Reservation,Federal reservation or trust land,5329.733273,30047940500,Montana,Lake County,5052,99999.0,0,1,1,POLYGON ((-113.9296866116276 47.53991224333398...
3,1110,Flathead Reservation,Federal reservation or trust land,5329.733273,30089940300,Montana,Sanders County,1879,99999.0,0,1,1,POLYGON ((-114.1873671952479 47.13766760917657...
4,1110,Flathead Reservation,Federal reservation or trust land,5329.733273,30047940400,Montana,Lake County,4262,99999.0,0,1,1,"POLYGON ((-113.9601981977866 47.6050154723254,..."


In [50]:
len(intersection)

643

In [51]:
dissolved = intersection.dissolve(by='geoid_tribal')

In [53]:
len(dissolved)

168

In [54]:
dissolved_trimmed = dissolved.reset_index()[[
    'geoid_tribal',
    'name_tribal',
    'geometry'
]]

In [55]:
dissolved_prepped = dissolved_trimmed.rename(columns={
    "geoid_tribal": "geoid",
    "name_tribal": "name"
})

In [56]:
dissolved_prepped.head()

,geoid,name,geometry
0,0010,Acoma Pueblo,"(POLYGON ((-107.777627 34.548139, -107.773395 ..."
1,0020,Agua Caliente Indian Reservation,(POLYGON ((-116.5822638967478 33.6851276989495...
2,0080,Allegany Reservation,"POLYGON ((-78.977519 41.999042, -78.977474 41...."
3,0110,Annette Island Reserve,"POLYGON ((-131.610929883141 54.9858807524663, ..."
4,0125,Augustine Reservation,"POLYGON ((-116.199282 33.656552, -116.197526 3..."


In [57]:
dissolved_prepped['area'] = dissolved_prepped.to_crs({'proj':'cea'}).geometry.map(lambda p: p.area / 10**6)

In [58]:
dissolved_prepped.head()

,geoid,name,geometry,area
0,0010,Acoma Pueblo,"(POLYGON ((-107.777627 34.548139, -107.773395 ...",1542.872627
1,0020,Agua Caliente Indian Reservation,(POLYGON ((-116.5822638967478 33.6851276989495...,0.480693
2,0080,Allegany Reservation,"POLYGON ((-78.977519 41.999042, -78.977474 41....",125.599965
3,0110,Annette Island Reserve,"POLYGON ((-131.610929883141 54.9858807524663, ...",550.594448
4,0125,Augustine Reservation,"POLYGON ((-116.199282 33.656552, -116.197526 3...",2.271942


In [60]:
dissolved_merged = dissolved_prepped.merge(
    reservations_deduped.rename(columns={"geoid_tribal": "geoid"})[['geoid', 'area']],
    on="geoid",
    suffixes=["_htc", "_total"]
)

In [61]:
dissolved_merged['pct_htc'] = dissolved_merged.area_htc / dissolved_merged.area_total

In [64]:
dissolved_merged.sort_values("area_htc", ascending=False).head(5)

,geoid,name,geometry,area_htc,area_total,pct_htc
86,2430,Navajo Nation Reservation,"(POLYGON ((-108.541557 35.034891, -108.54152 3...",62072.978786,62565.921391,0.992121
142,4200,Tohono O'odham Nation Reservation,"(POLYGON ((-112.719808 32.967355, -112.719534 ...",11534.902203,11534.986075,0.999993
18,0605,Cheyenne River Reservation,POLYGON ((-101.8392525976512 44.52395100593207...,11424.555187,11445.301110,0.998187
137,3970,Standing Rock Reservation,POLYGON ((-101.4702072145578 45.47181753760831...,9486.074003,9486.191063,0.999988
28,0845,Crow Reservation,"(POLYGON ((-107.013418 45.18194, -107.013418 4...",8327.919292,9343.671023,0.891290


In [65]:
dissolved_merged.sort_values("area_htc", ascending=False).tail(5)

,geoid,name,geometry,area_htc,area_total,pct_htc
97,2715,Pauma and Yuima Reservation,(POLYGON ((-116.9321159667102 33.3464051059000...,0.007937,24.252290,3.272689e-04
66,1850,La Jolla Reservation,(POLYGON ((-116.8410570065867 33.2559419054459...,0.002918,34.957758,8.348507e-05
15,0495,Capitan Grande Reservation,POLYGON ((-116.6811140223018 32.88451265870786...,0.002156,64.430007,3.346085e-05
146,4315,Tunica-Biloxi Reservation,(POLYGON ((-92.0626268708872 31.10577978474263...,0.000030,3.149779,9.520608e-06
94,2635,Pala Reservation,"POLYGON ((-117.000974922776 33.38656981010941,...",0.000006,52.706195,1.098324e-07


In [66]:
dissolved_merged.to_file("data/analysis/htc-reservations.geojson", driver="GeoJSON")

In [71]:
len(dissolved_merged)

168

In [67]:
dissolved_merged.area_htc.sum()

225788.0547851181

In [70]:
reservations_deduped['area'].sum()

295020.57328184653

In [72]:
dissolved_merged.area_htc.sum() / reservations_deduped['area'].sum()

0.7653298625022077